Compute the probability of census group having the rodent problems based on the records using the Bayesian Methods. Naive Bayesian will not work because our data are not discrete values, and making continuous variables into 

In [72]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import HillClimbSearch
from pgmpy.models import BayesianNetwork
import numpy as np

In [26]:
# loading the merged data 
df = pd.read_csv('..\\Data\\rodents_per_year_merged.csv')

In [27]:
# show the dummy variables
df.head()

,spatial_id,year,l_Commercial_sum,l_Other_sum,l_Outdoor_sum,l_Residential_sum,l_Residential-Mixed_sum,l_Vacant_Space_sum,d_Friday_sum,d_Monday_sum,...,c_Chinese_sum,c_Coffee/Tea_sum,c_Italian_sum,c_Japanese_sum,c_Latin American_sum,c_Mexican_sum,c_Pizza_sum,c_other_sum,num_violations,MedianHouseholdIncome
0,360050001000,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,360050001000,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,360050001000,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,360050001000,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,360050001000,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [28]:
# switch year to categorical
df['year'] = df['year'].astype('category')

# switch spatial id to categorical
df['spatial_id'] = df['spatial_id'].astype('category')

In [100]:
df['rodent_problem'] = df['num_sightings'].apply(lambda x: 1 if x > 0 else 0).astype('category')

In [101]:
# categorical variables
cat_vars = ['year', 'spatial_id', 'rodent_problem']

In [102]:
df = df.fillna(0)

In [103]:
# switching the rest columns to int except year, spatial_id, and rodent_problem
df.loc[:, df.columns.difference(cat_vars)] = df.loc[:, df.columns.difference(cat_vars)].astype('int')

In [104]:
df.dtypes

spatial_id                          category
year                                category
l_Commercial_sum                       int32
l_Other_sum                            int32
l_Outdoor_sum                          int32
l_Residential_sum                      int32
l_Residential-Mixed_sum                int32
l_Vacant_Space_sum                     int32
d_Friday_sum                           int32
d_Monday_sum                           int32
d_Saturday_sum                         int32
d_Sunday_sum                           int32
d_Thursday_sum                         int32
d_Tuesday_sum                          int32
d_Wednesday_sum                        int32
t_Evening_sum                          int32
t_Midday_sum                           int32
t_Morning_sum                          int32
num_sightings                          int32
s_Dead_Animal:Residential_sum          int32
s_Dead_Animal:Street_sum               int32
s_Dog_waste:Street_sum                 int32
s_Illegal_

In [105]:
# train test split
train, test = train_test_split(df, test_size=0.2) # 20% test data

# save the actual values of rodent problems
train_rodent = train['rodent_problem'] 
test_rodent = test['rodent_problem']

In [106]:
# train a bayesian net 
# find the best structure
hc = HillClimbSearch(train)
best_model = hc.estimate(scoring_method='BicScore')
print(best_model.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('year', 'v_rats/mice_sum'), ('year', 's_Trash_Unsecure:Residential_sum'), ('year', 's_Trash_Time:Street_sum'), ('year', 't_Midday_sum'), ('year', 's_Dead_Animal:Street_sum'), ('year', 'c_Coffee/Tea_sum'), ('year', 'c_Chinese_sum'), ('num_sightings', 'rodent_problem'), ('s_Trash_MissedService:Street_sum', 'year'), ('v_rats/mice_sum', 'num_violations'), ('v_rats/mice_sum', 'c_Pizza_sum'), ('v_rats/mice_sum', 'c_American_sum'), ('num_violations', 'v_flies_sum'), ('num_violations', 'c_other_sum'), ('num_violations', 'v_roaches_sum'), ('rodent_problem', 'l_Residential_sum'), ('rodent_problem', 't_Evening_sum'), ('rodent_problem', 't_Midday_sum'), ('rodent_problem', 'd_Monday_sum'), ('rodent_problem', 'd_Tuesday_sum'), ('rodent_problem', 'd_Wednesday_sum'), ('rodent_problem', 'd_Thursday_sum'), ('rodent_problem', 'd_Friday_sum'), ('rodent_problem', 'l_Other_sum'), ('rodent_problem', 'd_Saturday_sum'), ('rodent_problem', 'd_Sunday_sum'), ('rodent_problem', 'l_Residential-Mixed_sum'), ('rode

In [107]:
# train the model with the best structure
best_model = BayesianNetwork(hc.estimate(scoring_method='BicScore').edges())
best_model.fit(train, estimator=BayesianEstimator, prior_type="K2")

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [108]:
for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
for thenode in best_model.nodes():
    print(best_model.local_independencies(thenode))

CPD of year:
+----------------------------------+-----+
| s_Trash_MissedService:Street_sum | ... |
+----------------------------------+-----+
| year(2018)                       | ... |
+----------------------------------+-----+
| year(2019)                       | ... |
+----------------------------------+-----+
| year(2020)                       | ... |
+----------------------------------+-----+
| year(2021)                       | ... |
+----------------------------------+-----+
| year(2022)                       | ... |
+----------------------------------+-----+
| year(2023)                       | ... |
+----------------------------------+-----+
CPD of v_rats/mice_sum:
+---------------------+-----+-----------------------+
| year                | ... | year(2023)            |
+---------------------+-----+-----------------------+
| v_rats/mice_sum(0)  | ... | 0.745807424156774     |
+---------------------+-----+-----------------------+
| v_rats/mice_sum(1)  | ... | 0.1400037686074995

In [109]:
# print the nodes of the model 
best_model.nodes()

NodeView(('year', 'v_rats/mice_sum', 's_Trash_Unsecure:Residential_sum', 's_Trash_Time:Street_sum', 't_Midday_sum', 's_Dead_Animal:Street_sum', 'c_Coffee/Tea_sum', 'c_Chinese_sum', 'num_sightings', 'rodent_problem', 's_Trash_MissedService:Street_sum', 'num_violations', 'c_Pizza_sum', 'c_American_sum', 'v_flies_sum', 'c_other_sum', 'v_roaches_sum', 'l_Residential_sum', 't_Evening_sum', 'd_Monday_sum', 'd_Tuesday_sum', 'd_Wednesday_sum', 'd_Thursday_sum', 'd_Friday_sum', 'l_Other_sum', 'd_Saturday_sum', 'd_Sunday_sum', 'l_Residential-Mixed_sum', 'l_Commercial_sum', 'l_Vacant_Space_sum', 't_Morning_sum', 's_Trash:Street_sum', 'num_dsny_complaints', 's_Illegal_Dumping:Street_sum', 'l_Outdoor_sum', 's_Trash:Residential_sum', 's_Trash_Unsecure:Street_sum', 'subway_count', 's_Dog_waste:Street_sum', 'c_Mexican_sum', 's_Trash_Overflowing:Street_sum', 'c_Latin American_sum'))

In [110]:
# checking values in num_violations 
print(df.num_sightings.value_counts())

0      16068
1       7361
2       4244
3       2738
4       1794
       ...  
150        1
75         1
66         1
54         1
103        1
Name: num_sightings, Length: 74, dtype: int64


Need to find out which of the columns are directly related to the mice problems, so we are able to label the census groups. 

In [111]:
# print local independencies of rodent_problems
print(best_model.local_independencies('rodent_problem'))

In [112]:
# show what variables are in best_model 
best_model.nodes()

NodeView(('year', 'v_rats/mice_sum', 's_Trash_Unsecure:Residential_sum', 's_Trash_Time:Street_sum', 't_Midday_sum', 's_Dead_Animal:Street_sum', 'c_Coffee/Tea_sum', 'c_Chinese_sum', 'num_sightings', 'rodent_problem', 's_Trash_MissedService:Street_sum', 'num_violations', 'c_Pizza_sum', 'c_American_sum', 'v_flies_sum', 'c_other_sum', 'v_roaches_sum', 'l_Residential_sum', 't_Evening_sum', 'd_Monday_sum', 'd_Tuesday_sum', 'd_Wednesday_sum', 'd_Thursday_sum', 'd_Friday_sum', 'l_Other_sum', 'd_Saturday_sum', 'd_Sunday_sum', 'l_Residential-Mixed_sum', 'l_Commercial_sum', 'l_Vacant_Space_sum', 't_Morning_sum', 's_Trash:Street_sum', 'num_dsny_complaints', 's_Illegal_Dumping:Street_sum', 'l_Outdoor_sum', 's_Trash:Residential_sum', 's_Trash_Unsecure:Street_sum', 'subway_count', 's_Dog_waste:Street_sum', 'c_Mexican_sum', 's_Trash_Overflowing:Street_sum', 'c_Latin American_sum'))

In [113]:
# list nodes
lst_var = list(best_model.nodes())

# remove target rodent_problem
lst_var.remove('rodent_problem')

In [114]:
# making predictions using train data 
train = train[lst_var]
y_pred = best_model.predict(train)
y_pred

  0%|          | 0/26108 [00:00<?, ?it/s]

,rodent_problem
0,1
1,1
2,1
3,0
4,1
...,...
31161,1
31162,1
31163,0
31164,1


In [115]:
# comparing the predictions to the actual values in train data
train['pred_rodent_problem'] = y_pred['rodent_problem']
train['actual_rodent_problem'] = train_rodent

# fill na in pred_rodent_problem with 0
train['pred_rodent_problem'] = train['pred_rodent_problem'].fillna(0)

C:\Users\steve\AppData\Local\Temp\ipykernel_31032\2639478661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_rodent_problem'] = y_pred['rodent_problem']
C:\Users\steve\AppData\Local\Temp\ipykernel_31032\2639478661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['actual_rodent_problem'] = train_rodent
C:\Users\steve\AppData\Local\Temp\ipykernel_31032\2639478661.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [116]:
# in and out of sample accuracy using bayesian net
print('In sample accuracy:', (train['actual_rodent_problem'] == train['pred_rodent_problem']).sum()/train.shape[0])

In sample accuracy: 0.533305525251877


In [117]:
# making predictions using test data
test = test[lst_var]

In [118]:
# make predictions using test data, using for loop because some rows have index error 
# (might fix later in the future because it consumes great computational power)

test_pred = []
for i in range(len(test)):
    try:
        pred = best_model.predict(test.iloc[[i]])
        test_pred.append(pred.values[0])
    except IndexError:
        # append nan if index error
        test_pred.append(np.nan)
        print(f"Skipping row {i} due to an index error")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [119]:
# convert the array in the list to int in test_pred, else just append NaN
test_pred_i = [ int(x) if type(x) == np.ndarray else np.nan for x in test_pred ]

In [120]:
test_pred_i

[1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,


In [121]:
# shape of test_pred
len(test_pred_i)

7792

In [122]:
test

,year,v_rats/mice_sum,s_Trash_Unsecure:Residential_sum,s_Trash_Time:Street_sum,t_Midday_sum,s_Dead_Animal:Street_sum,c_Coffee/Tea_sum,c_Chinese_sum,num_sightings,s_Trash_MissedService:Street_sum,...,num_dsny_complaints,s_Illegal_Dumping:Street_sum,l_Outdoor_sum,s_Trash:Residential_sum,s_Trash_Unsecure:Street_sum,subway_count,s_Dog_waste:Street_sum,c_Mexican_sum,s_Trash_Overflowing:Street_sum,c_Latin American_sum
37778,2022,0,0,0,1,3,0,0,1,0,...,6,2,0,1,0,0,0,0,0,0
4349,2023,0,0,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,0,0
20497,2021,0,0,0,1,0,1,0,1,21,...,53,11,0,0,1,2,1,2,2,0
7376,2022,1,0,0,1,0,0,0,1,0,...,3,0,0,0,3,0,0,0,0,0
10497,2018,0,0,0,1,0,0,0,2,7,...,9,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17244,2020,0,0,0,1,5,0,0,1,8,...,18,1,0,2,0,0,0,0,0,0
36872,2022,1,0,0,2,6,0,1,2,0,...,20,6,0,0,0,0,1,0,0,0
27330,2020,0,0,0,0,1,0,0,1,0,...,2,0,0,0,0,0,0,0,0,0
15676,2019,3,0,0,0,2,0,1,0,7,...,29,6,0,1,1,1,1,0,0,0


In [125]:
# append the predictions to test
test['pred_rodent_problem'] = test_pred_i
test['actual_rodent_problem'] = test_rodent

Out of sample accuracy: 0.9964065708418891


In [128]:
# accuracy of test data
print('In sample accuracy:', (train['actual_rodent_problem'] == train['pred_rodent_problem']).sum()/train.shape[0])
print('Out of sample accuracy:', (test['actual_rodent_problem'] == test['pred_rodent_problem']).sum()/test.shape[0])

In sample accuracy: 0.533305525251877
Out of sample accuracy: 0.9964065708418891


In [ ]:
test.shape, train.shape

((7792, 45), (31166, 45))

The train accuracy does not fit the train set very well for Bayesian Network. This might be caused by the sparse data containing a large number of zeroes. We may analyze the rodent problems in NYC in other methoologies and may need additional data sources as well. 